In [1]:
import os
import json
import time
from pathlib import Path
from typing import List, Dict, Any, Tuple, Optional
from datetime import datetime

import numpy as np
import pandas as pd
import torch

from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification

from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFLoader

from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "")
if not OPENAI_API_KEY:
    raise ValueError("Set OPENAI_API_KEY in environment or .env file.")

# Configuration
EMBEDDING_MODEL = "text-embedding-3-small"
LLM_MODEL = "gpt-4o-mini"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Pipeline parameters
CHUNK_SIZE = 400
CHUNK_OVERLAP = 80
STAGE1_K = 30
TOP_K_RERANKED = 5

print(f"Using device: {DEVICE}")

/Users/ebpearls1/Desktop/Advanced-RAG-demo/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cpu


## Initialize Reranker Model

In [2]:
RERANKER_MODEL_NAME = "BAAI/bge-reranker-base"

tokenizer = AutoTokenizer.from_pretrained(RERANKER_MODEL_NAME)
reranker_model = AutoModelForSequenceClassification.from_pretrained(RERANKER_MODEL_NAME).to(DEVICE)

def cross_encoder_rerank(
    query: str,
    docs: List[Document],
    top_k: int = TOP_K_RERANKED
) -> List[Document]:
    if not docs:
        return []

    pairs = [(query, d.page_content) for d in docs]
    inputs = tokenizer(
        [p[0] for p in pairs],
        [p[1] for p in pairs],
        padding=True,
        truncation=True,
        return_tensors="pt",
        max_length=512
    ).to(DEVICE)

    with torch.no_grad():
        scores = reranker_model(**inputs).logits.squeeze(-1).cpu().numpy()

    ranked_idx = np.argsort(-scores)
    top_docs = [docs[i] for i in ranked_idx[:top_k]]
    return top_docs

## Helper Functions

In [3]:
def get_embedding_model() -> OpenAIEmbeddings:
    return OpenAIEmbeddings(model=EMBEDDING_MODEL, api_key=OPENAI_API_KEY)

def get_llm(model_name: str = LLM_MODEL, temperature: float = 0.7) -> ChatOpenAI:
    return ChatOpenAI(
        model=model_name,
        temperature=temperature,
        api_key=OPENAI_API_KEY,
    )

## Data Loading: SQuAD or PDF

In [4]:
def load_squad_subset(max_examples: int = 1000) -> Tuple[List[Document], pd.DataFrame]:
    """Load SQuAD dataset for testing."""
    ds = load_dataset("squad", split="train[:10%]")
    ds = ds.shuffle(seed=42).select(range(min(max_examples, len(ds))))

    contexts = []
    qa_rows = []

    for ex in ds:
        context = ex["context"]
        q = ex["question"]
        ans_texts = ex["answers"]["text"]
        ans = ans_texts[0] if ans_texts else ""

        contexts.append(context)
        qa_rows.append({
            "id": ex["id"],
            "context": context,
            "question": q,
            "answer": ans
        })

    unique_contexts = list({c: True for c in contexts}.keys())
    docs = [Document(page_content=c, metadata={"source": f"squad_paragraph_{i}"})
            for i, c in enumerate(unique_contexts)]

    qa_df = pd.DataFrame(qa_rows)
    return docs, qa_df

def load_pdf_documents(pdf_dir: str = "../data/pdfs") -> List[Document]:
    """Load PDF documents from a directory."""
    pdf_path = Path(pdf_dir)
    
    if not pdf_path.exists():
        print(f"Creating directory: {pdf_dir}")
        pdf_path.mkdir(parents=True, exist_ok=True)
        print(f"Please add PDF files to {pdf_dir} and run again.")
        return []
    
    pdf_files = list(pdf_path.glob("*.pdf"))
    
    if not pdf_files:
        print(f"No PDF files found in {pdf_dir}")
        return []
    
    docs = []
    for pdf_file in pdf_files:
        print(f"Loading: {pdf_file.name}")
        loader = PyPDFLoader(str(pdf_file))
        docs.extend(loader.load())
    
    print(f"Loaded {len(docs)} pages from {len(pdf_files)} PDF files")
    return docs

## Build Vectorstore

In [5]:
def build_vectorstore(
    docs: List[Document],
    chunk_size: int = CHUNK_SIZE,
    chunk_overlap: int = CHUNK_OVERLAP
) -> Tuple[FAISS, List[Document]]:
    """Build vectorstore with chunking."""
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n\n", "\n", ". ", " ", ""],
    )
    chunks = splitter.split_documents(docs)
    
    embeddings = get_embedding_model()
    vectordb = FAISS.from_documents(chunks, embedding=embeddings)
    return vectordb, chunks

## Feedback Memory System

In [6]:
class FeedbackMemory:
    """Stores user feedback and preferences to improve future responses."""
    
    def __init__(self, save_path: str = "../data/feedback_memory.json"):
        self.save_path = save_path
        self.feedback_history: List[Dict[str, Any]] = []
        self.preference_patterns: Dict[str, Any] = {
            "preferred_styles": [],
            "preferred_lengths": [],
            "preferred_structures": [],
            "context_preferences": []
        }
        self.load_memory()
    
    def add_feedback(
        self,
        query: str,
        response_a: str,
        response_b: str,
        preferred: str,
        context: str,
        reason: Optional[str] = None
    ):
        """Record user's preference between two responses."""
        feedback_entry = {
            "timestamp": datetime.now().isoformat(),
            "query": query,
            "response_a": response_a,
            "response_b": response_b,
            "preferred": preferred,  # "A" or "B"
            "context": context,
            "reason": reason,
            "response_a_length": len(response_a.split()),
            "response_b_length": len(response_b.split())
        }
        
        self.feedback_history.append(feedback_entry)
        self._update_preference_patterns(feedback_entry)
        self.save_memory()
    
    def _update_preference_patterns(self, feedback: Dict[str, Any]):
        """Analyze feedback to identify user preferences."""
        preferred_response = feedback["response_a"] if feedback["preferred"] == "A" else feedback["response_b"]
        preferred_length = feedback["response_a_length"] if feedback["preferred"] == "A" else feedback["response_b_length"]
        
        self.preference_patterns["preferred_lengths"].append(preferred_length)
        
        # Analyze style preferences
        if "detailed" in feedback.get("reason", "").lower() or len(preferred_response.split()) > 100:
            self.preference_patterns["preferred_styles"].append("detailed")
        elif "concise" in feedback.get("reason", "").lower() or len(preferred_response.split()) < 50:
            self.preference_patterns["preferred_styles"].append("concise")
    
    def get_preference_summary(self) -> Dict[str, Any]:
        """Get summary of user preferences."""
        if not self.feedback_history:
            return {"total_feedback": 0, "insights": "No feedback yet"}
        
        avg_length = np.mean(self.preference_patterns["preferred_lengths"]) if self.preference_patterns["preferred_lengths"] else 0
        
        style_counter = {}
        for style in self.preference_patterns["preferred_styles"]:
            style_counter[style] = style_counter.get(style, 0) + 1
        
        preferred_style = max(style_counter.items(), key=lambda x: x[1])[0] if style_counter else "balanced"
        
        return {
            "total_feedback": len(self.feedback_history),
            "avg_preferred_length": int(avg_length),
            "preferred_style": preferred_style,
            "style_distribution": style_counter
        }
    
    def get_generation_guidance(self) -> str:
        """Generate instruction for LLM based on learned preferences."""
        summary = self.get_preference_summary()
        
        if summary["total_feedback"] == 0:
            return ""
        
        guidance = "Based on user preferences: "
        
        if summary["avg_preferred_length"] < 50:
            guidance += "Keep responses concise and to the point. "
        elif summary["avg_preferred_length"] > 100:
            guidance += "Provide detailed, comprehensive responses. "
        
        if summary["preferred_style"] == "detailed":
            guidance += "Include explanations and background information. "
        elif summary["preferred_style"] == "concise":
            guidance += "Focus on direct answers without extra elaboration. "
        
        return guidance
    
    def save_memory(self):
        """Save feedback to disk."""
        os.makedirs(os.path.dirname(self.save_path), exist_ok=True)
        with open(self.save_path, 'w') as f:
            json.dump({
                "feedback_history": self.feedback_history,
                "preference_patterns": self.preference_patterns
            }, f, indent=2)
    
    def load_memory(self):
        """Load feedback from disk."""
        if os.path.exists(self.save_path):
            with open(self.save_path, 'r') as f:
                data = json.load(f)
                self.feedback_history = data.get("feedback_history", [])
                self.preference_patterns = data.get("preference_patterns", self.preference_patterns)
            print(f"Loaded {len(self.feedback_history)} feedback entries from memory")

# Initialize feedback memory
feedback_memory = FeedbackMemory()

## RAG Pipeline with Dual Response Generation

In [7]:
def retrieve_and_rerank(
    vectordb: FAISS,
    query: str,
    stage1_k: int = STAGE1_K,
    top_k: int = TOP_K_RERANKED
) -> List[Document]:
    """Retrieve and rerank documents."""
    candidates = vectordb.similarity_search(query, k=stage1_k)
    top_docs = cross_encoder_rerank(query, candidates, top_k=top_k)
    return top_docs

def generate_dual_responses(
    query: str,
    context_docs: List[Document],
    feedback_memory: FeedbackMemory
) -> Tuple[str, str, str]:
    """Generate two different responses for comparison."""
    
    context = "\n\n".join([d.page_content for d in context_docs])
    
    # Get learned preferences
    preference_guidance = feedback_memory.get_generation_guidance()
    
    # Response A: More detailed and explanatory
    llm_a = get_llm(temperature=0.7)
    system_msg_a = (
        "You are a helpful assistant that provides detailed, comprehensive answers. "
        "Use the context to give thorough explanations with examples and background information. "
        f"{preference_guidance}"
    )
    
    prompt_a = f"""Context:
\"\"\"{context}\"\"\"

Question: {query}

Provide a detailed, comprehensive answer based on the context:"""
    
    messages_a = [
        {"role": "system", "content": system_msg_a},
        {"role": "user", "content": prompt_a}
    ]
    response_a = llm_a.invoke(messages_a).content.strip()
    
    # Response B: More concise and direct
    llm_b = get_llm(temperature=0.3)
    system_msg_b = (
        "You are a helpful assistant that provides concise, direct answers. "
        "Use the context to give clear, to-the-point responses without unnecessary elaboration. "
        f"{preference_guidance}"
    )
    
    prompt_b = f"""Context:
\"\"\"{context}\"\"\"

Question: {query}

Provide a concise, direct answer based on the context:"""
    
    messages_b = [
        {"role": "system", "content": system_msg_b},
        {"role": "user", "content": prompt_b}
    ]
    response_b = llm_b.invoke(messages_b).content.strip()
    
    return response_a, response_b, context

## Interactive Query Interface

In [8]:
def display_responses(query: str, response_a: str, response_b: str):
    """Display both responses for comparison."""
    print("\n" + "="*80)
    print(f"QUERY: {query}")
    print("="*80)
    
    print("\n" + "-"*80)
    print("RESPONSE A (Detailed):")
    print("-"*80)
    print(response_a)
    print(f"\nLength: {len(response_a.split())} words")
    
    print("\n" + "-"*80)
    print("RESPONSE B (Concise):")
    print("-"*80)
    print(response_b)
    print(f"\nLength: {len(response_b.split())} words")
    print("\n" + "="*80)

def get_user_feedback() -> Tuple[str, Optional[str]]:
    """Get user's preferred response."""
    while True:
        choice = input("\nWhich response do you prefer? (A/B): ").strip().upper()
        if choice in ['A', 'B']:
            reason = input("Why did you prefer this response? (optional): ").strip()
            return choice, reason if reason else None
        print("Please enter 'A' or 'B'")

def run_interactive_query(
    vectordb: FAISS,
    query: str,
    feedback_memory: FeedbackMemory
):
    """Run a single query with dual responses and feedback collection."""
    print(f"\n🔍 Processing query: {query}")
    
    # Retrieve documents
    top_docs = retrieve_and_rerank(vectordb, query)
    print(f"✓ Retrieved {len(top_docs)} relevant documents")
    
    # Generate dual responses
    print("✓ Generating two response variants...")
    response_a, response_b, context = generate_dual_responses(query, top_docs, feedback_memory)
    
    # Display responses
    display_responses(query, response_a, response_b)
    
    # Get feedback
    preferred, reason = get_user_feedback()
    
    # Store feedback
    feedback_memory.add_feedback(
        query=query,
        response_a=response_a,
        response_b=response_b,
        preferred=preferred,
        context=context,
        reason=reason
    )
    
    print(f"\n✓ Feedback recorded! The system will learn from your preference.")
    
    # Show preference summary
    summary = feedback_memory.get_preference_summary()
    print(f"\n📊 Preference Summary: {summary['total_feedback']} feedback entries collected")
    if summary['total_feedback'] > 0:
        print(f"   - Preferred style: {summary['preferred_style']}")
        print(f"   - Average preferred length: {summary['avg_preferred_length']} words")

## Load Data and Build Index

Choose your data source: SQuAD dataset or PDF documents

In [9]:
# Configuration: Choose data source
DATA_SOURCE = "squad"  # Options: "squad" or "pdf"
PDF_DIRECTORY = "../data/pdfs"  # Directory containing PDF files

print(f"Data source: {DATA_SOURCE}")

if DATA_SOURCE == "squad":
    # Load SQuAD dataset
    base_docs, qa_df = load_squad_subset(max_examples=600)
    print(f"Loaded {len(base_docs)} documents from SQuAD")
    
elif DATA_SOURCE == "pdf":
    # Load PDF documents
    base_docs = load_pdf_documents(PDF_DIRECTORY)
    if not base_docs:
        print("\n⚠️  No PDF documents found. Please add PDFs to the directory.")
    else:
        print(f"Loaded {len(base_docs)} pages from PDF files")

else:
    raise ValueError("DATA_SOURCE must be 'squad' or 'pdf'")

# Build vectorstore
if base_docs:
    print("\nBuilding vectorstore...")
    vectordb, chunks = build_vectorstore(base_docs)
    print(f"✓ Vectorstore built with {len(chunks)} chunks")
else:
    vectordb = None
    print("⚠️  No documents to index")

Data source: squad
Loaded 496 documents from SQuAD

Building vectorstore...
Loaded 496 documents from SQuAD

Building vectorstore...
✓ Vectorstore built with 1272 chunks
✓ Vectorstore built with 1272 chunks


## Run Interactive Pipeline

Test the pipeline with example queries

In [10]:
# Example queries (modify based on your data)
if DATA_SOURCE == "squad":
    example_queries = [
        "What is the capital of France?",
        "Who invented the telephone?",
        "When did World War II end?"
    ]
else:
    example_queries = [
        "What is the main topic of this document?",
        "Summarize the key findings.",
        "What are the main recommendations?"
    ]

print("Example queries available:")
for i, q in enumerate(example_queries, 1):
    print(f"{i}. {q}")

print("\nYou can use these or enter your own queries below.")

Example queries available:
1. What is the capital of France?
2. Who invented the telephone?
3. When did World War II end?

You can use these or enter your own queries below.


### Query 1: First Example

In [11]:
if vectordb is not None:
    # Use first example query or modify it
    query1 = example_queries[0]
    run_interactive_query(vectordb, query1, feedback_memory)


🔍 Processing query: What is the capital of France?
✓ Retrieved 5 relevant documents
✓ Generating two response variants...
✓ Retrieved 5 relevant documents
✓ Generating two response variants...

QUERY: What is the capital of France?

--------------------------------------------------------------------------------
RESPONSE A (Detailed):
--------------------------------------------------------------------------------
The capital of France is Paris. Paris is not only the political capital of the country but also serves as a major cultural, economic, and historical hub in Europe and the world. 

As mentioned in the context, Paris is significant in various industries, including finance, retail, tourism, and the arts. It is home to many iconic landmarks such as the Eiffel Tower, the Louvre Museum, and the Notre-Dame Cathedral, which attract millions of tourists each year. The city's influence extends beyond its borders, as it is often considered a center for fashion and art, hosting events l

### Query 2: Second Example

In [12]:
if vectordb is not None:
    # Use second example query or modify it
    query2 = example_queries[1]
    run_interactive_query(vectordb, query2, feedback_memory)


🔍 Processing query: Who invented the telephone?
✓ Retrieved 5 relevant documents
✓ Generating two response variants...
✓ Retrieved 5 relevant documents
✓ Generating two response variants...

QUERY: Who invented the telephone?

--------------------------------------------------------------------------------
RESPONSE A (Detailed):
--------------------------------------------------------------------------------
The context provided does not mention the invention of the telephone. However, the telephone is commonly attributed to Alexander Graham Bell, who was awarded the first US patent for an "improvement in telegraphy" in 1876, which allowed for the transmission of voice over wires. Bell's invention marked a significant advancement in communication technology, enabling real-time voice conversations over long distances.

While the context references telecommunications and devices inspired by existing designs, it does not directly address the history or inventor of the telephone. For a co

### Query 3: Third Example

In [13]:
if vectordb is not None:
    # Use third example query or modify it
    query3 = example_queries[2]
    run_interactive_query(vectordb, query3, feedback_memory)


🔍 Processing query: When did World War II end?
✓ Retrieved 5 relevant documents
✓ Generating two response variants...
✓ Retrieved 5 relevant documents
✓ Generating two response variants...

QUERY: When did World War II end?

--------------------------------------------------------------------------------
RESPONSE A (Detailed):
--------------------------------------------------------------------------------
World War II ended on September 2, 1945, when Japan formally surrendered. This followed the earlier surrender of Germany on May 7, 1945, which marked the end of the war in Europe. The conclusion of World War II had significant global implications, including the establishment of new political orders, the onset of the Cold War, and the emergence of the United States as a dominant economic power, as indicated by the context provided. The post-war period saw substantial economic growth, influenced in part by returning veterans and government policies aimed at stimulating the economy.

L

### Custom Query

Enter your own query below

In [14]:
if vectordb is not None:
    # Enter your custom query here
    custom_query = "Where is Normandy?"
    
    # Uncomment the line below to run with your custom query
    run_interactive_query(vectordb, custom_query, feedback_memory)


🔍 Processing query: Where is Normandy?
✓ Retrieved 5 relevant documents
✓ Generating two response variants...
✓ Retrieved 5 relevant documents
✓ Generating two response variants...

QUERY: Where is Normandy?

--------------------------------------------------------------------------------
RESPONSE A (Detailed):
--------------------------------------------------------------------------------
The context provided does not contain information about Normandy. However, I can provide a detailed answer about its location.

Normandy is a region located in the northwestern part of France. It is bordered by the English Channel to the north, the regions of Bretagne (Brittany) to the west, Pays de la Loire to the southwest, Centre-Val de Loire to the southeast, and Île-de-France to the east. Normandy is known for its rich history, including its role in the D-Day landings during World War II, and is famous for landmarks such as the picturesque Mont Saint-Michel and the historic city of Rouen.

If 

## Analyze Feedback Patterns

In [15]:
# View all feedback history
if feedback_memory.feedback_history:
    feedback_df = pd.DataFrame(feedback_memory.feedback_history)
    
    print("\n📊 Feedback Analysis")
    print("="*80)
    
    # Preference distribution
    pref_counts = feedback_df['preferred'].value_counts()
    print(f"\nPreference Distribution:")
    print(f"  Response A (Detailed): {pref_counts.get('A', 0)} times")
    print(f"  Response B (Concise): {pref_counts.get('B', 0)} times")
    
    # Average lengths
    print(f"\nAverage Response Lengths:")
    print(f"  Response A: {feedback_df['response_a_length'].mean():.1f} words")
    print(f"  Response B: {feedback_df['response_b_length'].mean():.1f} words")
    
    # Preferred lengths
    preferred_a_lengths = feedback_df[feedback_df['preferred'] == 'A']['response_a_length']
    preferred_b_lengths = feedback_df[feedback_df['preferred'] == 'B']['response_b_length']
    all_preferred_lengths = pd.concat([preferred_a_lengths, preferred_b_lengths])
    
    if len(all_preferred_lengths) > 0:
        print(f"\nPreferred Response Characteristics:")
        print(f"  Average length: {all_preferred_lengths.mean():.1f} words")
        print(f"  Length range: {all_preferred_lengths.min():.0f} - {all_preferred_lengths.max():.0f} words")
    
    # Show recent feedback
    print(f"\n📝 Recent Feedback (last 5):")
    print("-"*80)
    for entry in feedback_memory.feedback_history[-5:]:
        print(f"\nQuery: {entry['query'][:60]}...")
        print(f"Preferred: Response {entry['preferred']}")
        if entry.get('reason'):
            print(f"Reason: {entry['reason']}")
    
    # Display dataframe
    print("\n" + "="*80)
    display(feedback_df[['timestamp', 'query', 'preferred', 'response_a_length', 'response_b_length', 'reason']].tail(10))
    
else:
    print("No feedback collected yet. Run some queries first!")


📊 Feedback Analysis

Preference Distribution:
  Response A (Detailed): 1 times
  Response B (Concise): 3 times

Average Response Lengths:
  Response A: 137.5 words
  Response B: 7.8 words

Preferred Response Characteristics:
  Average length: 29.8 words
  Length range: 6 - 94 words

📝 Recent Feedback (last 5):
--------------------------------------------------------------------------------

Query: What is the capital of France?...
Preferred: Response B
Reason: It's a direct and short answer to my direct question.

Query: Who invented the telephone?...
Preferred: Response B
Reason: The context should be used.

Query: When did World War II end?...
Preferred: Response A
Reason: It explains the history which I prefer

Query: Where is Normandy?...
Preferred: Response B
Reason: Context matters



,timestamp,query,preferred,response_a_length,response_b_length,reason
0,2025-11-20T17:10:41.483012,What is the capital of France?,B,215,6,It's a direct and short answer to my direct qu...
1,2025-11-20T17:11:38.867262,Who invented the telephone?,B,115,11,The context should be used.
2,2025-11-20T17:12:23.777813,When did World War II end?,A,94,6,It explains the history which I prefer
3,2025-11-20T17:13:24.682253,Where is Normandy?,B,126,8,Context matters


## View Learned Preferences

In [16]:
summary = feedback_memory.get_preference_summary()
guidance = feedback_memory.get_generation_guidance()

print("\n🎯 Learned User Preferences")
print("="*80)
print(f"Total feedback collected: {summary['total_feedback']}")

if summary['total_feedback'] > 0:
    print(f"\nPreferred Style: {summary['preferred_style'].upper()}")
    print(f"Average Preferred Length: {summary['avg_preferred_length']} words")
    
    if summary.get('style_distribution'):
        print(f"\nStyle Distribution:")
        for style, count in summary['style_distribution'].items():
            print(f"  {style.capitalize()}: {count} times")
    
    print(f"\n📋 Current Generation Guidance:")
    print(f"  {guidance}")
    
    print("\n✨ The system will use these preferences to improve future responses!")
else:
    print("\nNo preferences learned yet. Provide feedback to help the system learn!")


🎯 Learned User Preferences
Total feedback collected: 4

Preferred Style: CONCISE
Average Preferred Length: 29 words

Style Distribution:
  Concise: 3 times

📋 Current Generation Guidance:
  Based on user preferences: Keep responses concise and to the point. Focus on direct answers without extra elaboration. 

✨ The system will use these preferences to improve future responses!


## Export Feedback Data

In [17]:
# Export feedback to CSV for further analysis
if feedback_memory.feedback_history:
    output_path = "../data/feedback_export.csv"
    feedback_df = pd.DataFrame(feedback_memory.feedback_history)
    feedback_df.to_csv(output_path, index=False)
    print(f"✓ Feedback data exported to: {output_path}")
    print(f"  Total entries: {len(feedback_df)}")
else:
    print("No feedback to export")

✓ Feedback data exported to: ../data/feedback_export.csv
  Total entries: 4


## Continuous Query Loop (Optional)

Run this cell to enter a continuous loop for multiple queries

In [ ]:
def run_continuous_loop(vectordb: FAISS, feedback_memory: FeedbackMemory):
    """Run continuous query loop until user stops."""
    print("\n🔄 Continuous Query Mode")
    print("="*80)
    print("Enter 'quit' or 'exit' to stop\n")
    
    while True:
        query = input("\nEnter your question: ").strip()
        
        if query.lower() in ['quit', 'exit', 'q']:
            print("\n✓ Exiting continuous mode")
            break
        
        if not query:
            print("Please enter a valid question")
            continue
        
        try:
            run_interactive_query(vectordb, query, feedback_memory)
        except KeyboardInterrupt:
            print("\n\n✓ Interrupted by user")
            break
        except Exception as e:
            print(f"\n❌ Error: {e}")
            continue

# Uncomment to run continuous loop
# if vectordb is not None:
#     run_continuous_loop(vectordb, feedback_memory)

## Summary

This notebook implements:
1. ✅ Dual data source support (SQuAD + PDF)
2. ✅ Dual response generation (detailed vs concise)
3. ✅ User feedback collection
4. ✅ Preference learning and adaptation
5. ✅ Memory-based improvement over time

The system learns from your preferences and adapts future responses accordingly!